# Groq + Google Workspace MCP: Intelligent Email, Calendar & Drive Automation

This notebook demonstrates how to empower Groq inference with Google Workspace capabilities using Model Context Protocol (MCP) connectors for Gmail, Google Calendar, and Google Drive. By connecting Groq's ultra-fast inference with Google's productivity suite, you can build AI agents that read emails, schedule meetings, search documents, and manage your workspace—all through natural language.

## What are Google Workspace MCP Connectors?

Google Workspace MCP connectors provide AI models with secure, authenticated access to your productivity data. Instead of being limited to training data, your AI can:

- **Access real-time data** from Gmail, Calendar, and Drive
- **Take actions** like sending emails or creating calendar events  
- **Coordinate across services** in a single natural language request
- **Maintain security** through OAuth 2.0 authentication

These connectors use the industry-standard OAuth 2.0 flow, ensuring your data remains secure and access is properly scoped.

## What We'll Build

We'll create an AI productivity assistant that:

1. **Searches** your Gmail for unread emails and provides intelligent summaries
2. **Checks** your Google Calendar to view schedules and find meetings
3. **Searches** your Google Drive for documents based on natural language queries
4. **Coordinates** across all three services to provide comprehensive workspace insights

This showcases how Google Workspace MCP connectors enable powerful productivity automation through simple, fast AI interactions.

---


In [ ]:
%pip install openai python-dotenv ipython

## Getting Started

Follow these steps to set up:

### 1. Get your Groq API key
**Sign up** for Groq at [console.groq.com](https://console.groq.com/keys) to get your free API key.


### 2. Get your Google Workspace OAuth Token

Google Workspace MCP connectors require OAuth 2.0 authentication. You'll need to obtain an access token with the following **required scopes**:


#### Required OAuth Scopes

**From Gmail API v1:**
- `https://www.googleapis.com/auth/gmail.modify` - Read and modify Gmail messages

**From Google Calendar API v3:**
- `https://www.googleapis.com/auth/calendar.events` - Manage calendar events

**From Google Drive API v3:**
- `https://www.googleapis.com/auth/drive.readonly` - Download the content of Drive files

**From Google OAuth2 API v2:**
- `https://www.googleapis.com/auth/userinfo.email` - View your email address
- `https://www.googleapis.com/auth/userinfo.profile` - View your basic profile info


**To obtain your OAuth token:**
1. Visit the [OAuth Playground](https://developers.google.com/oauthplayground/)
2. In Step 1, select **all five scopes** listed above
3. Click **"Authorize APIs"** and sign in with your Google account
4. In Step 2, click **"Exchange authorization code for tokens"**
5. Copy the **Access token** (starts with `ya29.`)

**Note:** OAuth tokens expire after approximately 1 hour. You'll need to refresh them for extended sessions.


### 3. Configure your environment
**Paste your API keys** into the cell below and run the cell.

*Note: do **not** run the cell below if your keys are already configured in an .env file in this directory*


In [ ]:
# To export your API keys into a .env file, run the following cell (replace with your actual keys):
!echo "GROQ_API_KEY=<your-groq-api-key>" >> .env
!echo "GSUITE_ACCESS_TOKEN=<your-oauth-access-token>" >> .env

In [ ]:
import json
import os
import time
from datetime import datetime
from dotenv import load_dotenv

from openai import OpenAI
from IPython.display import Markdown

load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")
GSUITE_ACCESS_TOKEN = os.getenv("GSUITE_ACCESS_TOKEN")

if not GROQ_API_KEY:
    print("Please set your Groq API key")
else:
    print("✅ Groq API key configured successfully!")
if not GSUITE_ACCESS_TOKEN:
    print("Please set your Google Workspace OAuth token")
else:
    print("✅ Google Workspace OAuth token configured successfully!")
    
MODEL = "openai/gpt-oss-120b"

## Step 1: Set up the Groq client


In [ ]:
client = OpenAI(base_url="https://api.groq.com/api/openai/v1", api_key=GROQ_API_KEY)

## Step 2: Set up Google Workspace MCP Connectors

Unlike remote MCP servers that use `server_url`, Google Workspace uses **connectors** with `connector_id` and OAuth `authorization`. This allows for secure, user-authenticated access to your Google Workspace data.

You can configure one or more connectors depending on which Google services you want to access:


In [ ]:
# Configure Google Workspace MCP connectors
# You can use one or multiple connectors based on your needs

tools = [
    # Gmail connector for email operations
    {
        "type": "mcp",
        "server_label": "gmail",
        "connector_id": "connector_gmail",
        "authorization": GSUITE_ACCESS_TOKEN,
        "require_approval": "never",
    },
    # Google Calendar connector for calendar operations
    {
        "type": "mcp",
        "server_label": "google_calendar",
        "connector_id": "connector_googlecalendar",
        "authorization": GSUITE_ACCESS_TOKEN,
        "require_approval": "never",
    },
    # Google Drive connector for document operations
    {
        "type": "mcp",
        "server_label": "googledrive",
        "connector_id": "connector_googledrive",
        "authorization": GSUITE_ACCESS_TOKEN,
        "require_approval": "never",
    },
]

## Step 3: Connect Groq to Google Workspace MCPs through Groq's Responses API

This will be our **main** function we use to call the Groq API with Google Workspace connectors.


In [ ]:
def call_groq_with_gsuite(client, tools, query):
    """
    Connect Groq to Google Workspace through MCP connectors.

    This function demonstrates the speed and accuracy of combining:
    - Groq's fast LLM inference (500+ tokens/second)
    - Google Workspace MCP connectors for email, calendar, and drive access
    """

    start_time = time.time()

    response = client.responses.create(
        model=MODEL,
        input=query,
        tools=tools,
        stream=False,
        temperature=0.1,
        top_p=0.4,
    )

    total_time = time.time() - start_time

    content = (
        response.output_text if hasattr(response, "output_text") else str(response)
    )

    executed_tools = []

    if hasattr(response, "output") and response.output:
        for output_item in response.output:
            if hasattr(output_item, "type") and output_item.type == "mcp_call":
                executed_tools.append(
                    {
                        "type": "mcp",
                        "arguments": getattr(output_item, "arguments", "{}"),
                        "output": getattr(output_item, "output", ""),
                        "name": getattr(output_item, "name", ""),
                        "server_label": getattr(output_item, "server_label", ""),
                    }
                )
    return {
        "content": content,
        "total_time": total_time,
        "mcp_calls_performed": executed_tools,
        "timestamp": datetime.now().isoformat(),
    }

Let's implement a helper function to display MCP tool calls and their results. This will provide transparency into which tools were called, their arguments, and outputs.

(This is generally useful when debugging if you need to view raw MCP outputs, it's not vital for functionality)


In [ ]:
def print_mcp_calls(mcp_calls):
    executed_tools = mcp_calls["mcp_calls_performed"]
    if executed_tools:
        print(f"\nGOOGLE WORKSPACE MCP CALLS: Found {len(executed_tools)} tool call(s):")
        print("-" * 50)
        for i, tool in enumerate(executed_tools, 1):
            print(f"\nTool Call #{i}")
            print(f"   Type: {tool['type']}")
            print(f"   Tool Name: {tool['name']}")
            print(f"   Connector: {tool['server_label']}")
            try:
                if tool["arguments"]:
                    args = (
                        json.loads(tool["arguments"])
                        if isinstance(tool["arguments"], str)
                        else tool["arguments"]
                    )
                    print(f"   Arguments: {args}")

                if tool["output"]:
                    output_data = (
                        json.loads(tool["output"])
                        if isinstance(tool["output"], str)
                        else tool["output"]
                    )
                    # Truncate long outputs for readability
                    if isinstance(output_data, dict):
                        if "messages" in output_data:
                            print(f"   Found {len(output_data['messages'])} message(s)")
                        elif "events" in output_data:
                            print(f"   Found {len(output_data['events'])} event(s)")
                        elif "files" in output_data:
                            print(f"   Found {len(output_data['files'])} file(s)")
                        else:
                            print(f"   Output Preview: {str(output_data)[:200]}...")
                    else:
                        print(f"   Output: {str(output_data)[:200]}...")
            except Exception as e:
                print(f"   Could not parse tool data: {e}")
    print(f"\n   Total time: {mcp_calls['total_time']:.2f} seconds")
    print(f"   Google Workspace MCP calls: {len(mcp_calls['mcp_calls_performed'])}")

# Examples

**Note:** Some queries may consume more tokens than others depending on the amount of tool calls the model makes. Please be aware of various rate limits that are tied to your API keys if you happen to run into any rate limit errors.

**Important:** Make sure your OAuth token has the necessary scopes for the operations you want to perform.

---


## Demo 1: Gmail - Search Unread Emails

Let's start with a simple Gmail operation: searching for recent unread emails. This demonstrates how Groq can quickly process your email data and provide intelligent summaries.


In [ ]:
# Search for latest unread emails
gmail_search = call_groq_with_gsuite(
    client,
    tools,
    "Search for my latest 3 unread emails and provide a summary of each",
)


Let's display the agent's response in markdown format.


In [ ]:
Markdown(gmail_search["content"])

Let's examine the agent's intermediate steps, including how it calls Gmail connector tools.


In [ ]:
print_mcp_calls(gmail_search)

## Demo 2: Google Calendar - Check Today's Schedule

Now let's interact with Google Calendar to retrieve today's events and get an overview of your schedule.


In [ ]:
calendar_check = call_groq_with_gsuite(
    client,
    tools,
    "What's on my calendar for today? Give me a summary of all my meetings.",
)


Let's display the calendar summary.


In [ ]:
Markdown(calendar_check["content"])

Let's examine the agent's intermediate steps.


In [ ]:
print_mcp_calls(calendar_check)

## Demo 3: Google Drive - Search Documents

Let's use Google Drive to search for specific documents and get information about them.


In [ ]:
drive_search = call_groq_with_gsuite(
    client,
    tools,
    "Search my Google Drive for documents containing 'quarterly report' and list the most recently modified ones",
)


Let's display the search results.


In [ ]:
Markdown(drive_search["content"])

Let's examine the agent's intermediate steps.


## Demo 4: Google Drive - Find PDF Documents

Let's search specifically for PDF files in your Drive and get their details. The `drive.readonly` scope allows us to download and read the content of Drive files.

In [ ]:
# Search for PDF files
pdf_search = call_groq_with_gsuite(
    client,
    tools,
    "List PDF files in my Google Drive and show me the 3 most recent ones with their names and sizes.",
)


Let's display the PDF search results.

In [ ]:
Markdown(pdf_search["content"])

Let's examine the agent's intermediate steps.

In [ ]:
print_mcp_calls(pdf_search)

## Demo 5: Multi-Service Workflow - Email & Calendar Integration

Now let's demonstrate the power of combining multiple Google Workspace services. We'll search for emails about meetings and cross-reference them with the calendar.


In [ ]:
multi_service = call_groq_with_gsuite(
    client,
    tools,
    "Check my Gmail for unread emails and my calendar for today's meetings. Give me a brief summary of both.",
)


Let's display the cross-referenced results.


In [ ]:
Markdown(multi_service["content"])

Let's examine the agent's intermediate steps.


In [ ]:
print_mcp_calls(multi_service)

## Demo 6: Try it Yourself

Now it's your turn! Create your own Google Workspace automation by replacing the query below with your specific task.

### Safe ideas for screen recording:
- **Email stats**: "How many unread emails do I have? Group them by sender domain (e.g., gmail.com, work domain)"
- **Calendar summary**: "How many meetings do I have this week? What's my busiest day?"
- **Drive overview**: "How many files do I have in my Drive? What are the most common file types?"
- **File organization**: "List my 5 largest files in Drive with their sizes"
- **Weekly summary**: "Give me a count of emails, calendar events, and Drive files for this week"

### Advanced ideas (use with your own data):
- **Email triage**: "Find all emails from my boss this week and summarize the key action items"
- **Meeting prep**: "What meetings do I have tomorrow? For each meeting, search my Drive for related documents"
- **Document search**: "Search my Drive for files containing 'budget' or 'proposal'"


In [ ]:
your_query = "Your Query Here"  # Change this!

custom_response = call_groq_with_gsuite(client, tools, your_query)


In [ ]:
Markdown(custom_response["content"])

In [ ]:
print_mcp_calls(custom_response)

## Available Google Workspace MCP Tools

The Google Workspace MCP connectors provide access to a wide range of tools across Gmail, Google Calendar, and Google Drive:

### Gmail Tools
| Tool | Description |
|------|-------------|
| **`gmail_search`** | Search for emails with advanced filters (sender, date, labels, etc.) |
| **`gmail_get_message`** | Retrieve full details of a specific email message |
| **`gmail_send`** | Send emails with attachments and formatting |
| **`gmail_create_draft`** | Create draft emails to review before sending |
| **`gmail_modify_labels`** | Add or remove labels from messages |
| **`gmail_list_labels`** | Get all available labels in your Gmail account |

### Google Calendar Tools
| Tool | Description |
|------|-------------|
| **`calendar_list_events`** | List events within a specific time range |
| **`calendar_get_event`** | Get detailed information about a specific event |
| **`calendar_create_event`** | Create new calendar events with attendees and reminders |
| **`calendar_update_event`** | Modify existing calendar events |
| **`calendar_delete_event`** | Remove events from your calendar |
| **`calendar_list_calendars`** | List all calendars you have access to |

### Google Drive Tools
| Tool | Description |
|------|-------------|
| **`drive_search_files`** | Search for files and folders with advanced queries |
| **`drive_get_file`** | Get metadata and content of a specific file |
| **`drive_list_files`** | List files in a specific folder or across Drive |
| **`drive_create_file`** | Create new files or folders |
| **`drive_update_file`** | Modify existing file metadata or content |
| **`drive_delete_file`** | Move files to trash |
| **`drive_export_file`** | Export Google Docs/Sheets/Slides to standard formats |

### Key Benefits

1. **Natural Language Interface**: Control Google Workspace using plain English
2. **Ultra-Fast Responses**: Sub-second inference from Groq combined with real-time Google data
3. **Multi-Service Coordination**: Seamlessly work across Gmail, Calendar, and Drive in one workflow
4. **Secure OAuth Authentication**: Industry-standard OAuth 2.0 for safe access to your data
5. **Full Transparency**: See exactly which tools were called and what data was accessed

---


## Alternative: Using curl

Prefer curl? Here's a quick example for Gmail:

```bash
curl -X POST https://api.groq.com/api/openai/v1/responses \
  -H "Authorization: Bearer YOUR_GROQ_API_KEY" \
  -H "Content-Type: application/json" \
  -d '{"model":"openai/gpt-oss-20b","tools":[{"type":"mcp","server_label":"gmail","connector_id":"connector_gmail","authorization":"YOUR_OAUTH_TOKEN","require_approval":"never"}],"input":"Show me my last 3 unread emails"}'
```

Replace connector details for Calendar (`connector_googlecalendar`) or Drive (`connector_googledrive`).

---


## Summary

You've just experienced the power of combining **Groq's fast inference** with **Google Workspace MCP connectors**.

## 🔧 **Google Workspace MCP Connectors**

**Description**: Google Workspace MCP connectors enable AI models to interact with Gmail, Google Calendar, and Google Drive through secure OAuth authentication. Build intelligent assistants that can read emails, manage your schedule, search documents, and automate your productivity workflows.

### What Makes This Powerful:

1. **Speed + Intelligence**: Groq's ultra-fast inference means your AI assistant responds instantly to complex workspace queries
2. **Real-Time Data**: Unlike static training data, MCP connectors give your model access to your actual, current emails, calendar, and files
3. **Multi-Service Workflows**: Coordinate actions across multiple Google services in a single natural language request
4. **Secure Authentication**: OAuth 2.0 ensures your data is accessed securely with proper scope limitations
5. **Productivity Automation**: Build custom workflows for email triage, meeting prep, document search, and more

### Use Cases:

- **Executive Assistant**: Summarize important emails, manage calendar, and prepare for meetings
- **Email Triage**: Automatically categorize, prioritize, and respond to emails
- **Meeting Intelligence**: Search for meeting-related documents and provide context before calls
- **Document Discovery**: Find and organize files across your Drive based on natural language queries
- **Schedule Optimization**: Analyze calendar patterns and suggest better time management
- **Cross-Service Workflows**: Connect email threads with calendar events and relevant documents

---

**Resources:**
- 🔑 **OAuth 2.0 Playground**: [developers.google.com/oauthplayground](https://developers.google.com/oauthplayground/)
- 📚 **Google Workspace APIs**: [developers.google.com/workspace](https://developers.google.com/workspace)
- 🚀 **Groq Console**: [console.groq.com](https://console.groq.com)

---

**Happy automating with Groq + Google Workspace! 🚀**